***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0 --root-user-action=ignore
!pip install -q graphframes --root-user-action=ignore

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage  # Import the storage module

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 31 06:35 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '315347757work3ir'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab.

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

# מחזיר רשימה של כל המילים בצורה הבאה : [(key:token , value:[(doc_id,tf),(doc_id,tf)])] RDD
def word_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    filtered_tokens = [token for token in tokens if token not in all_stopwords]
    tf_dict = {}
    for token in filtered_tokens:
        if token in tf_dict:
            tf_dict[token] += 1
        else:
            tf_dict[token] = 1
    return [(token, (id, tf)) for token, tf in tf_dict.items()]

# לכל value מהרשימה הקודמת הוא מבצע מיון לפי doc_id
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0])


def calculate_df(postings):
    return postings.mapValues(lambda posting_list: len(posting_list))


def partition_postings_and_write(postings):
    # Partition postings into buckets using token2bucket_id
    bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))

    # Group by bucket ID to get all posting lists in each bucket
    grouped_postings = bucketed_postings.groupByKey()

    # Step 3: Process each bucket, convert ResultIterable to a list, and write to disk
    posting_locations = grouped_postings.map(lambda b_w_pl: InvertedIndex.write_a_posting_list(b_w_pl,bucket_name))
    return posting_locations


In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
index_const_time

3397.397433280945

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 18.45 MiB  2025-01-01T08:47:07Z  gs://315347757work3ir/postings_gcp/index.pkl
TOTAL: 1 objects, 19344222 bytes (18.45 MiB)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
    # Extract edges: (source_id, destination_id)
    edges = pages.flatMap(lambda row: [(row['id'], link['id']) for link in row['anchor_text']])

    # Remove duplicate edges
    edges = edges.distinct()

    # Extract vertices: all unique IDs from both sources and destinations
    vertices = edges.flatMap(lambda edge: [edge[0], edge[1]]).distinct()

    return edges, vertices

In [ ]:

t_start = time()
pages_links = spark.read.parquet("gs://315347757work3ir/multistream*").select("id", "anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
# vertices_tuples = vertices.map(lambda x: (x,))
# verticesDF = vertices.toDF(['id']).repartition(124, 'id')
verticesDF = vertices.map(lambda x: (x,)).toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

25/01/01 10:38:09 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_83_22 !
25/01/01 10:38:09 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_83_64 !
25/01/01 10:38:09 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_180_110 !
25/01/01 10:38:09 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_98_1 !
25/01/01 10:38:09 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_83_74 !
25/01/01 10:38:09 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_98_99 !
25/01/01 10:38:10 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_180_109 !
25/01/01 10:38:10 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_88_110 !
25/01/01 10:38:10 WARN org.apache.spark.stor

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750|  9913.72878216078|
|  10568| 5385.349263642037|
|  32927| 5282.081575765277|
|  30680|  5128.23370960412|
|5843419|  4957.56768626387|
|  68253|  4769.27826535516|
|  31717|  4486.35018054831|
|  11867|  4146.41465091277|
|  14533|3996.4664408855033|
| 645042| 3531.627089803744|
|  17867|3246.0983906041406|
|5042916| 2991.945739166179|
|4689264|2982.3248830417488|
|  14532| 2934.746829203171|
|  25391|2903.5462235133973|
|   5405|2891.4163291546356|
|4764461| 2834.366987332661|
|  15573|2783.8651181588384|
|   9316|2782.0396464137702|
|8569916|2775.2861918400167|
+-------+------------------+
only showing top 20 rows



In [ ]:

# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120

7130.495651960373

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete
the reporting requirements for assignment #3.

In [ ]:
# size of input data
!gsutil du -sh "gs://315347757work3ir/*"

20.27 GiB    gs://315347757work3ir/*


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://315347757work3ir/postings_gcp


In [2]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 50
print(f'I used {cost} USD credit during the course of this assignment')

I used 50 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour.


In [ ]:
#If you have decided to do the bonus task - please copy the code here

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag